# RFP RAG w/o LangChain Demo

#### Pre-requisites

In [ ]:
%pip install -q qdrant-client

#### Load the existing questions

In [ ]:
# load documents
import os
from csv import DictReader
from uuid import uuid4

## grab all csv files with the name  "rfp_existing_questions_<client_name>.csv"
documents = []
root_dir = "datasets/rag/"
for file in os.listdir(root_dir):
    if file.startswith("rfp_existing_questions_") and file.endswith(".csv"):
        # use csv dict reader to load the csv file
        with open(os.path.join(root_dir, file)) as f:
            reader = DictReader(f)
            for row in reader:
                # add a unique id to the row
                row["id"] = str(uuid4())
                documents.append(row)
print(len(documents))

#### Generate embeddings for the questions

In [ ]:
# load openai api key
from dotenv import load_dotenv

load_dotenv()

# now we can import and use the openai client
from openai import OpenAI

client = OpenAI()


def get_embedding(text):
    """Returns a text embedding for the given text"""
    return client.embeddings.create(
        input=text,
        model="text-embedding-3-small",
    ).data[0].embedding

In [ ]:
from tqdm import tqdm

for doc in tqdm(documents):
    doc["embedding_question"] = get_embedding(doc["RFP_Question"])
    doc["embedding_answer"] = get_embedding(doc["RFP_Answer"])

#### Setup Vector DB

In [ ]:
from qdrant_client import QdrantClient

qdrant = QdrantClient(":memory:")

#### Insert embeddings and questions into Vector DB

In [ ]:
from qdrant_client.models import Distance, PointStruct, VectorParams

qdrant.recreate_collection(
    "my_rag_collection",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

qdrant.upsert(
    "my_rag_collection",
    points=[
        PointStruct(
            id=doc["id"],
            vector=doc["embedding_question"],
            payload={
                k: v
                for k, v in doc.items()
                if k not in ["embedding_question", "embedding_answer"]
            },
        )
        for doc in documents
    ],
)

#### Load the new questions

In [ ]:
# load new questions from `rfp_new_questions_client_100.csv`
new_questions = []
root_dir = "datasets/rag/"
for file in os.listdir(root_dir):
    if file.startswith("rfp_new_questions_") and file.endswith(".csv"):
        # use csv dict reader to load the csv file
        with open(os.path.join(root_dir, file)) as f:
            reader = DictReader(f)
            for row in reader:
                # add a unique id to the row
                row["id"] = str(uuid4())
                new_questions.append(row)

#### Create `query` function for Vector DB

In [ ]:
def query(question, limit=10, _print=True):
    """Returns the top 10 results for the given question. If _print is True, it will print the results."""
    response = qdrant.search(
        "my_rag_collection",
        query_vector=get_embedding(question),
        limit=limit,
    )

    if _print:
        print(f"For the question:\n\"{new_questions[0]['RFP_Question']}\"\n")
        print(f"Top 10 results are:\n")
        for result in response:
            print(f"Score: {result.score} (higher is better)")
            print(f"Question: {result.payload['RFP_Question']}")
            print(f"Answer: {result.payload['RFP_Answer']}\n")

    return response

#### Test the `query` function

In [ ]:
response = query(new_questions[0]["RFP_Question"])

#### Setup Pipeline that takes question, queries Vector DB and generates an answer with LLM

In [ ]:
system_prompt = """
You are an expert RFP AI assistant.
You are tasked with answering new RFP questions based on existing RFP questions and answers.
You will be provided with the existing RFP questions and answer pairs that are the most relevant to the new RFP question.
After that you will be provided with a new RFP question.
You will generate an answer and respond only with the answer.
""".strip()


def generate_answer(question):
    """Generates an answer for the given question using RAG and an LLM."""
    context = ""
    results = query(new_questions[0]["RFP_Question"], _print=False)
    for result in results:
        context += f"Q: {result.payload['RFP_Question']}\nA: {result.payload['RFP_Answer']}"
        context += f"Project: {result.payload['Project_Title']}\n\n"

    print(f"For Question: \"{question}\"\n")
    print(f"Context:\n```\n{context}```\n")

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": context},
            {"role": "user", "content": question},
        ],
    )

    answer = response.choices[0].message.content

    print(f"Answer:\n{answer}")

    return answer

#### Test Pipeline

In [ ]:
answer = generate_answer(new_questions[0]["RFP_Question"])